# OpenAI Function Calling 101
Một trong những khó khăn khi sử dụng các mô hình ngôn ngữ lớn (LLM) như ChatGPT là chúng không tạo ra đầu ra dữ liệu có cấu trúc. Điều này rất quan trọng đối với các hệ thống lập trình phụ thuộc phần lớn vào dữ liệu có cấu trúc để tương tác. Ví dụ, nếu bạn muốn xây dựng một chương trình phân tích cảm xúc của một bài đánh giá phim, bạn có thể phải thực hiện một đoạn code trông giống như sau:

```
prompt = f'''
Please perform a sentiment analysis on the following movie review:
{MOVIE_REVIEW_TEXT}
Please output your response as a single word: either "Positive" or "Negative". Do not add any extra characters.
'''
```

Vấn đề là điều này không phải lúc nào cũng hiệu quả. Khá phổ biến là LLM sẽ thêm vào một dấu chấm không mong muốn hoặc giải thích dài hơn như: "Cảm xúc của bộ phim này là: Tích cực." Mặc dù bạn có thể sử dụng biểu thức chính quy (regex) để lấy ra câu trả lời (🤢), nhưng rõ ràng đây không phải là lý tưởng. Điều lý tưởng sẽ là nếu LLM trả về kết quả dưới dạng cấu trúc JSON như sau:

```
{
    'sentiment': 'positive'
}
```

OpenAI đã giới thiệu một tính năng mới gọi là function calling, giúp giải quyết vấn đề trên. Function calling chính là câu trả lời cho vấn đề trên. Jupyter notebook này sẽ minh họa một ví dụ đơn giản về cách sử dụng function calling mới của OpenAI trong Python. Nếu bạn muốn xem tài liệu đầy đủ, [vui lòng xem liên kết này](https://platform.openai.com/docs/guides/gpt/function-calling).

## Notebook Setup
Hãy bắt đầu với các import. Bây giờ, có thể bạn đã cài đặt client Python `openai`, nhưng rất có thể bạn cần nâng cấp nó để có được chức năng function calling mới. Đây là cách nâng cấp trong Terminal / Powershell của bạn bằng `pip`:

```
pip install openai --upgrade
```

In [ ]:
!pip install groq --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.2 MB/s eta 0:00:00


In [ ]:
# Importing the necessary Python libraries
from IPython.display import clear_output
import os
import json
import yaml
from groq import Groq

Để kiểm tra chức năng function calling, tôi đã viết một đoạn "About Me" ngắn chứa các sự thật cụ thể mà chúng ta sẽ phân tích thành các cấu trúc dữ liệu phù hợp, bao gồm số nguyên và chuỗi. Hãy tải văn bản này vào

In [ ]:
from google.colab import userdata


# Loading the "About Me" text from local file
os.environ['GROQ_API_KEY'] = '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'
about_me = "Hello! My name is David Hundley. I am a principal machine learning engineer at State Farm. I enjoy learning about AI and teaching what I learn back to others. I have two daughters. I drive a Tesla Model 3, and my favorite video game series is The Legend of Zelda."

print(about_me)


Hello! My name is David Hundley. I am a principal machine learning engineer at State Farm. I enjoy learning about AI and teaching what I learn back to others. I have two daughters. I drive a Tesla Model 3, and my favorite video game series is The Legend of Zelda.


## The Pre-Function Calling Days
Trước khi chúng ta minh họa function calling, hãy minh họa cách chúng ta đã sử dụng prompt engineering và Regex để tạo ra một JSON có cấu trúc mà chúng ta có thể làm việc một cách lập trình sau này.

In [ ]:
# Engineering a prompt to extract as much information from "About Me" as a JSON object
about_me_prompt = f'''
Please extract information as a JSON object. Please look for the following pieces of information.
Name
Job title
Company
Number of children as a single number
Car make
Car model
Favorite video game series

This is the body of text to extract the information from:
{about_me}
'''

In [ ]:
client = Groq(
    api_key = os.environ.get('GROQ_API_KEY')
)

groq_response = client.chat.completions.create(
    model = 'llama3-8b-8192',
    messages = [{'role': 'user', 'content': about_me_prompt}]
)

In [ ]:
# Loading the response as a JSON object
json_response = json.loads(groq_response.choices[0].message.content)
print(json_response)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
groq_response.choices[0].message.content

'Here is the extracted information in a JSON object:\n\n```\n{\n  "name": "David Hundley",\n  "jobTitle": "Principal Machine Learning Engineer",\n  "company": "State Farm",\n  "children": 2,\n  "carMake": "Tesla",\n  "carModel": "Model 3",\n  "favoriteVideoGameSeries": "The Legend of Zelda"\n}\n```\n\nLet me know if you need anything else!'

## Using the New Function Calling Capabilities
Bây giờ chúng ta đã minh họa cách chúng ta đã từng lấy được JSON có cấu trúc trong những ngày trước khi có function calling, hãy chuyển sang cách chúng ta có thể sử dụng function calling để trích xuất các kết quả tương tự nhưng theo cách nhất quán hơn cho việc sử dụng hệ thống của chúng ta. Chúng ta sẽ bắt đầu đơn giản hơn với một hàm tùy chỉnh duy nhất và sau đó giải quyết một số chức năng "nâng cao" hơn.

In [ ]:
# Defining our initial extract_person_info function
def extract_person_info(name, job_title, num_children):
    '''
    Prints basic "About Me" information

    Inputs:
        name (str): Name of the person
        job_title (str): Job title of the person
        num_chilren (int): The number of children the parent has.
    '''

    print(f'This person\'s name is {name}. Their job title is {job_title}, and they have {num_children} children.')

In [ ]:
# Defining how we want ChatGPT to call our custom functions
my_custom_functions = [
    {
        'name': 'extract_person_info',
        'description': 'Get "About Me" information from the body of the input text',
        'parameters': {
            'type': 'object',
            'properties': {
                'name': {
                    'type': 'string',
                    'description': 'Name of the person'
                },
                'job_title': {
                    'type': 'string',
                    'description': 'Job title of the person'
                },
                'num_children': {
                    'type': 'integer',
                    'description': 'Number of children the person is a parent to'
                }
            }
        }
    }
]

In [ ]:
# Getting the response back from ChatGPT (gpt-3.5-turbo)
groq_response = client.chat.completions.create(
    model = 'llama3-8b-8192',
    messages = [{'role': 'user', 'content': about_me}],
    functions = my_custom_functions,
    function_call = 'auto'
)

print(groq_response)

ChatCompletion(id='chatcmpl-ac6ef877-7808-4ede-bbf5-05a0b031e887', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"name":"David Hundley","job_title":"Principal Machine Learning Engineer","num_children":2}', name='extract_person_info'), tool_calls=None))], created=1728857394, model='llama3-8b-8192', object='chat.completion', system_fingerprint='fp_af05557ca2', usage=CompletionUsage(completion_tokens=88, prompt_tokens=1021, total_tokens=1109, completion_time=0.073333333, prompt_time=0.115240994, queue_time=0.002358924999999998, total_time=0.188574327), x_groq={'id': 'req_01ja3ywjvdf55am0tb21q4gbxc'})


In [ ]:
print(groq_response.choices[0].message.function_call)

FunctionCall(arguments='{"name":"David Hundley","job_title":"Principal Machine Learning Engineer","num_children":2}', name='extract_person_info')


### What if the prompt I submit doesn't contain the information I want to extract per my custom function?
Copy code
Trong ví dụ ban đầu của chúng ta, hàm tùy chỉnh đã tìm cách trích xuất ba thông tin rất cụ thể và chúng ta đã chứng minh rằng điều này hoạt động thành công bằng cách truyền văn bản "About Me" tùy chỉnh của tôi dưới dạng một prompt. Nhưng bạn có thể tự hỏi, điều gì sẽ xảy ra nếu bạn truyền vào bất kỳ prompt nào khác không chứa thông tin đó?

Hãy nhớ lại rằng chúng ta đã đặt một tham số trong lệnh gọi API client của mình gọi là function_call mà chúng ta đặt thành auto. Chúng ta sẽ khám phá điều này sâu hơn nữa trong phần tiếp theo, nhưng về cơ bản, tham số này đang yêu cầu ChatGPT sử dụng phán đoán tốt nhất của nó để tìm ra khi nào cần cấu trúc đầu ra cho một trong các hàm tùy chỉnh của chúng ta.

Vậy điều gì sẽ xảy ra khi chúng ta gửi một prompt không khớp với bất kỳ hàm tùy chỉnh nào của chúng ta? Nói một cách đơn giản, nó sẽ mặc định trở lại hành vi thông thường như thể function calling không tồn tại. Hãy kiểm tra điều này với một prompt tùy ý: "Tháp Eiffel cao bao nhiêu?"

In [ ]:

groq_response3 = client.chat.completions.create(
    model = 'llama3-8b-8192',
    messages = [{'role': 'user', 'content': 'How tall is the Eiffel Tower?'}],
    functions = my_custom_functions,
    function_call = 'auto'
)

print(groq_response3)

BadRequestError: Error code: 400 - {'error': {'message': "Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': '<tool-use>\n{\n\t"tool_calls": [\n\t\t{\n\t\t\t"id": "pending",\n\t\t\t"type": "function",\n\t\t\t"function": {\n\t\t\t\t"name": "extract_info"\n\t\t\t},\n\t\t\t"parameters": {\n\t\t\t\t"questions": {\n\t\t\t\t\t"text": "What is the height of the Eiffel Tower in meters?"\n\t\t\t\t}\n\t\t\t}\n\t\t}\n\t]\n}\n</tool-use>'}}

In [ ]:
# Defining a function to extract only vehicle information
def extract_vehicle_info(vehicle_make, vehicle_model):
    '''
    Prints basic vehicle information

    Inputs:
        - vehicle_make (str): Make of the vehicle
        - vehicle_model (str): Model of the vehicle
    '''

    print(f'Vehicle make: {vehicle_make}\nVehicle model: {vehicle_model}')



# Defining a function to extract all information provided in the original "About Me" prompt
def extract_all_info(name, job_title, num_children, vehicle_make, vehicle_model, company_name, favorite_vg_series):
    '''
    Prints the full "About Me" information

    Inputs:
        - name (str): Name of the person
        - job_title (str): Job title of the person
        - num_chilren (int): The number of children the parent has
        - vehicle_make (str): Make of the vehicle
        - vehicle_model (str): Model of the vehicle
        - company_name (str): Name of the company the person works for
        - favorite_vg_series (str): Person's favorite video game series.
    '''

    print(f'''
    This person\'s name is {name}. Their job title is {job_title}, and they have {num_children} children.
    They drive a {vehicle_make} {vehicle_model}.
    They work for {company_name}.
    Their favorite video game series is {favorite_vg_series}.
    ''')

In [ ]:
# Defining how we want ChatGPT to call our custom functions
my_custom_functions = [
    {
        'name': 'extract_person_info',
        'description': 'Get "About Me" information from the body of the input text',
        'parameters': {
            'type': 'object',
            'properties': {
                'name': {
                    'type': 'string',
                    'description': 'Name of the person'
                },
                'job_title': {
                    'type': 'string',
                    'description': 'Job title of the person'
                },
                'num_children': {
                    'type': 'integer',
                    'description': 'Number of children the person is a parent to'
                }
            }
        }
    },
    {
        'name': 'extract_vehicle_info',
        'description': 'Extract the make and model of the person\'s car',
        'parameters': {
            'type': 'object',
            'properties': {
                'vehicle_make': {
                    'type': 'string',
                    'description': 'Make of the person\'s vehicle'
                },
                'vehicle_model': {
                    'type': 'string',
                    'description': 'Model of the person\'s vehicle'
                }
            }
        }
    },
    {
        'name': 'extract_all_info',
        'description': 'Extract all information about a person including their vehicle make and model',
        'parameters': {
            'type': 'object',
            'properties': {
                'name': {
                    'type': 'string',
                    'description': 'Name of the person'
                },
                'job_title': {
                    'type': 'string',
                    'description': 'Job title of the person'
                },
                'num_children': {
                    'type': 'integer',
                    'description': 'Number of children the person is a parent to'
                },
                'vehicle_make': {
                    'type': 'string',
                    'description': 'Make of the person\'s vehicle'
                },
                'vehicle_model': {
                    'type': 'string',
                    'description': 'Model of the person\'s vehicle'
                },
                'company_name': {
                    'type': 'string',
                    'description': 'Name of the company the person works for'
                },
                'favorite_vg_series': {
                    'type': 'string',
                    'description': 'Name of the person\'s favorite video game series'
                }
            }
        }
    }
]

Now let's demonstrate what happens when we apply 3 different samples to all of the custom functions.

In [ ]:
# Defining a list of samples
samples = [
    about_me,
    'My name is David Hundley. I am a principal machine learning engineer, and I have two daughters.',
    'She drives a Kia Sportage.'
]

In [ ]:
# Iterating over the three samples
for i, sample in enumerate(samples):

    print(f'Sample #{i + 1}\'s results:')

    # Getting the response back from ChatGPT (gpt-3.5-turbo)
    groq_response1 = client.chat.completions.create(
        model = 'llama3-8b-8192',
        messages = [{'role': 'user', 'content': sample}],
        functions = my_custom_functions,
        function_call = 'auto'
    )

    # Printing the sample's response
    print(groq_response1)

Sample #1's results:
ChatCompletion(id='chatcmpl-94b63aa5-7350-4225-a3d2-7ff1fa501743', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"name":"David Hundley","num_children":2}', name='extract_person_info'), tool_calls=None))], created=1728857770, model='llama3-8b-8192', object='chat.completion', system_fingerprint='fp_6a6771ae9c', usage=CompletionUsage(completion_tokens=77, prompt_tokens=1429, total_tokens=1506, completion_time=0.064166667, prompt_time=0.203900812, queue_time=0.0021255980000000063, total_time=0.268067479), x_groq={'id': 'req_01ja3z826ge6vsjkx56hmqx5a7'})
Sample #2's results:
ChatCompletion(id='chatcmpl-54e72624-a4db-46f3-a0d0-ae356ab5aaee', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"name":"David Hundley","num_children"

With each of the respective prompts, ChatGPT selected the correct custom function, and we can specifically note that in the `name` value under `function_call` in the API's response object. In addition to this being a handy way to identify which function to use the arguments for, we can programmatically map our actual custom Python function to this value to run the correct code appropriately.

In [ ]:
# Iterating over the three samples
for i, sample in enumerate(samples):

    print(f'Sample #{i + 1}\'s results:')

    # Getting the response back from ChatGPT (gpt-3.5-turbo)
    openai_response = client.chat.completions.create(
        model = 'llama3-8b-8192',
        messages = [{'role': 'user', 'content': sample}],
        functions = my_custom_functions,
        function_call = 'auto'
    ).choices[0].message

    # Checking to see that a function call was invoked
    if openai_response.function_call:
        # Checking to see which specific function call was invoked
        function_called = openai_response.function_call.name

        # Extracting the arguments of the function call
        function_args = json.loads(openai_response.function_call.arguments)

        # Invoking the proper functions
        if function_called == 'extract_person_info':
            extract_person_info(*list(function_args.values()))
        elif function_called == 'extract_vehicle_info':
            extract_vehicle_info(*list(function_args.values()))
        elif function_called == 'extract_all_info':
            extract_all_info(*list(function_args.values()))

Sample #1's results:
This person's name is David Hundley. Their job title is principal machine learning engineer, and they have 2 children.
Sample #2's results:
This person's name is Principal Machine Learning Engineer. Their job title is David Hundley, and they have 2 children.
Sample #3's results:


BadRequestError: Error code: 400 - {'error': {'message': "Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': '<tool-use>\n{\n\t"tool_calls": [\n\t\t{\n\t\t\t"id": "pending",\n\t\t\t"type": "function",\n\t\t\t"function": {\n\t\t\t\t"name": "extract_person_info"\n\t\t\t},\n\t\t\t"parameters": {\n\t\t\t\t"num_children": "0",\n\t\t\t\t"name": "Unknown",\n\t\t\t\t"job_title": "Unknown"\n\t\t\t}\n\t\t}\n\t]\n}\n</tool-use>'}}

## OpenAI Function Calling with LangChain
Given its popularity amongst the Generative AI community, I thought I'd re-visit this notebook and add some code to show how you might make use of this exact same functionality in LangChain

In [ ]:
!pip install --quiet -U langchain langchain-groq langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.4 MB/s eta 0:00:00


In [ ]:
# Importing the LangChain objects
from langchain_groq import ChatGroq
from langchain.chains import LLMChain
from langchain.prompts.chat import ChatPromptTemplate
from langchain.chains.openai_functions import create_structured_output_chain

In [ ]:
# Setting the proper instance of the OpenAI model
llm = ChatGroq(model = 'llama3-8b-8192')

# Setting a LangChain ChatPromptTemplate
chat_prompt_template = ChatPromptTemplate.from_template('{my_prompt}')

# Setting the JSON schema for extracting vehicle information
langchain_json_schema = {
    'name': 'extract_vehicle_info',
    'description': 'Extract the make and model of the person\'s car',
    'type': 'object',
    'properties': {
        'vehicle_make': {
            'title': 'Vehicle Make',
            'type': 'string',
            'description': 'Make of the person\'s vehicle'
        },
        'vehicle_model': {
            'title': 'Vehicle Model',
            'type': 'string',
            'description': 'Model of the person\'s vehicle'
        }
    }
}

In [ ]:
# Defining the LangChain chain object for function calling
chain = create_structured_output_chain(output_schema = langchain_json_schema,
                                       llm = llm,
                                       prompt = chat_prompt_template)

<ipython-input-42-0d6661282edd>:2: LangChainDeprecationWarning: The function `create_structured_output_chain` was deprecated in LangChain 0.1.1 and will be removed in 1.0. Use :meth:`~ChatOpenAI.with_structured_output` instead.
  chain = create_structured_output_chain(output_schema = langchain_json_schema,


In [ ]:
# Getting results with a demo prompt
print(chain.invoke(input = {'my_prompt': 'I drive a Tesla Model 3'}))

{'my_prompt': 'I drive a Tesla Model 3', 'function': {'extract_vehicle_info': {'vehicle_make': 'Tesla', 'vehicle_model': 'Model 3'}}}


In [ ]:
clear_output()